In [20]:
# Import required libraries
import requests
import pandas as pd
import json

from api_keys import geoapify_api_key

In [4]:
stadiums_data = pd.read_csv('Data/Stadiums_All.csv')

In [5]:
stadiums_data.head()

,ID,City,Club,Stadium,Cap,Country
0,67,Barnsley,Barnsley,Oakwell Stadium,"23,287",England
1,68,Birmingham,Aston Villa,Villa Park,"42,785",England
2,69,Birmingham,Birmingham City,St Andrew's Stadium,"30,009",England
3,70,Blackburn,Blackburn Rovers,Ewood Park,"31,154",England
4,71,Blackpool,Blackpool,Bloomfield Road,"16,007",England


In [12]:
stadiums_data.shape

(241, 6)

In [17]:
stadiums_data_df=stadiums_data.dropna(how='any')

In [18]:
stadiums_data_df.head()

,ID,City,Club,Stadium,Cap,Country
0,67,Barnsley,Barnsley,Oakwell Stadium,"23,287",England
1,68,Birmingham,Aston Villa,Villa Park,"42,785",England
2,69,Birmingham,Birmingham City,St Andrew's Stadium,"30,009",England
3,70,Blackburn,Blackburn Rovers,Ewood Park,"31,154",England
4,71,Blackpool,Blackpool,Bloomfield Road,"16,007",England


In [25]:
# list of stadiums
stadiums = stadiums_data_df['Stadium']


In [26]:
# Your Geoapify API key
api_key = geoapify_api_key


In [27]:
# Base URL for the API
base_url = 'https://api.geoapify.com/v1/geocode/search'

In [28]:
# Empty list to store results
results = []

In [29]:
# Loop over all stadiums
for stadium in stadiums:
    # Parameters for the API request
    params = {
        'text': stadium,
        'apiKey': api_key
    }

    # Make the request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response as JSON
        data = response.json()

        # Check if any results were found
        if data['features']:
            # Get the first result
            result = data['features'][0]

            # Get the coordinates (longitude, latitude) of the result
            coordinates = result['geometry']['coordinates']

            # Add the coordinates to the results list
            results.append({
                'Stadium': stadium,
                'Longitude': coordinates[0],
                'Latitude': coordinates[1]
            })
        else:
            print(f"No results found for {stadium}")
    else:
        print(f"Request failed with status code {response.status_code}")


In [32]:
# Convert the results to a DataFrame
df = pd.DataFrame(results)

In [33]:
# Display the DataFrame
print(df)

                           Stadium  Longitude   Latitude
0                  Oakwell Stadium  -1.467725  53.552140
1                       Villa Park -87.977837  41.889312
2              St Andrew's Stadium  -1.868041  52.475567
3                       Ewood Park  -2.489258  53.728609
4                  Bloomfield Road  -2.373736  51.365293
..                             ...        ...        ...
224          Estadio Jose Zorrilla  -4.761258  41.644580
225            Estadio El Madrigal  -0.103405  39.944208
226  Estadio Municipal de Balaidos  -8.739231  42.211122
227        Estadio de Mendizorroza  -9.260969  38.708812
228            Estadio La Romareda  -0.901823  41.636489

[229 rows x 3 columns]


In [35]:
final_stadium_df=pd.merge(stadiums_data_df,df,how='left',on='Stadium')
final_stadium_df.head()

,ID,City,Club,Stadium,Cap,Country,Longitude,Latitude
0,67,Barnsley,Barnsley,Oakwell Stadium,"23,287",England,-1.467725,53.552140
1,68,Birmingham,Aston Villa,Villa Park,"42,785",England,-87.977837,41.889312
2,69,Birmingham,Birmingham City,St Andrew's Stadium,"30,009",England,-1.868041,52.475567
3,70,Blackburn,Blackburn Rovers,Ewood Park,"31,154",England,-2.489258,53.728609
4,71,Blackpool,Blackpool,Bloomfield Road,"16,007",England,-2.373736,51.365293


In [41]:
final_stadium_df=final_stadium_df.drop('ID',axis=1)

In [42]:
final_stadium_df.head()

,City,Club,Stadium,Cap,Country,Longitude,Latitude
0,Barnsley,Barnsley,Oakwell Stadium,"23,287",England,-1.467725,53.552140
1,Birmingham,Aston Villa,Villa Park,"42,785",England,-87.977837,41.889312
2,Birmingham,Birmingham City,St Andrew's Stadium,"30,009",England,-1.868041,52.475567
3,Blackburn,Blackburn Rovers,Ewood Park,"31,154",England,-2.489258,53.728609
4,Blackpool,Blackpool,Bloomfield Road,"16,007",England,-2.373736,51.365293


In [43]:
final_stadium_df.to_csv('Data/Stadium_Lat_Lng_data.csv', index=False)